In [1]:
import pandas as pd
import geopandas
from folium import Circle, Marker, Icon, Map
from pymongo import MongoClient, GEOSPHERE
conn = MongoClient("localhost:27017")
db = conn.get_database("madrid")
import random
from folium import Circle, Marker, Icon, Map

In [2]:
df_restaurants = pd.read_csv("data/restaurants.csv", index_col=0)
df_snacks = pd.read_csv("data/snacks.csv", index_col=0)
df_culture = pd.read_csv("data/culture.csv", index_col=0)
df_party = pd.read_csv("data/party.csv", index_col=0)
df_drinks = pd.read_csv("data/drinks.csv", index_col=0)
df_outdoors = pd.read_csv("data/outdoors.csv", index_col=0)
df_leisure = pd.read_csv("data/leisure.csv", index_col=0)

In [3]:
#list(df_restaurants.CP.unique())

In [4]:
leisure = db.get_collection("leisure")
outdoors = db.get_collection("outdoors")
restaurants = db.get_collection("restaurants")
drinks = db.get_collection("drinks")
party = db.get_collection("party")
snacks = db.get_collection("snacks")
culture = db.get_collection("culture")
transport = db.get_collection("transport")


## Team-building

In [5]:
db.list_collection_names()

['culture',
 'transport',
 'todo',
 'leisure',
 'party',
 'restaurants',
 'drinks',
 'snacks',
 'outdoors']

In [6]:
intersection = set(list(df_leisure.CP.unique())).intersection(set(list(df_restaurants.CP.unique())))

In [7]:
cp = list(intersection)

In [8]:
cp_random = random.choice(cp)

In [9]:
leis = list(leisure.find({"CP":str(int(cp_random))}))

In [10]:
rest = list(restaurants.find({"CP":str(int(cp_random))}))

In [11]:
l_random = random.choice(leis)
r_random = random.choice(rest)

In [12]:
l_random

{'_id': ObjectId('60a0f5a8ca2d1425dcbbdbd9'),
 'name': 'Bowling Ilusiona Plenilunio',
 'rating': 4.3,
 'address': 'C.C. Plenilunio, Calle de Aracne, 3, 28022 Madrid, Spain',
 'price_level': 1.0,
 'place': 'bowling',
 'CP': '28022',
 'latitude': 40.4475017,
 'longitude': -3.5868256,
 'geometry': {'type': 'Point', 'coordinates': [-3.5868256, 40.4475017]},
 'price': '1-15€'}

In [13]:
r_random

{'_id': ObjectId('60a0f5a7bd9ddfe838fcbff9'),
 'name': 'The Good Burger',
 'rating': 3.9,
 'address': 'Calle de Aracne, S/N, 28022 Madrid, Spain',
 'price_level': 1.0,
 'place': 'burger',
 'CP': '28022',
 'latitude': 40.4473406,
 'longitude': -3.5866978,
 'geometry': {'type': 'Point', 'coordinates': [-3.5866978, 40.4473406]},
 'price': '1-15€'}

## Friends

In [14]:
intersection = set(list(df_drinks.CP.unique())).intersection(set(list(df_party.CP.unique())))

cp = list(intersection)

cp_random = random.choice(cp)

drinks_ = list(drinks.find({"CP":str(int(cp_random))}))

party_ = list(party.find({"CP":str(int(cp_random))}))

d_random = random.choice(drinks_)
p_random = random.choice(party_)

In [15]:
d_random

{'_id': ObjectId('60a0f5a7b68aeec84a076b84'),
 'name': 'TomeVinos',
 'rating': 4.7,
 'address': 'Paseo de la Chopera, 194, 28100 Alcobendas, Madrid, Spain',
 'price_level': 2.0,
 'place': 'wine',
 'CP': '28100',
 'latitude': 40.5453377,
 'longitude': -3.6590494,
 'geometry': {'type': 'Point', 'coordinates': [-3.6590494, 40.5453377]},
 'price': '1-15€'}

In [16]:
p_random

{'_id': ObjectId('60a0f5a85976320580e34311'),
 'name': 'Sala Liverpool Karaoke',
 'rating': 3.4,
 'address': 'Plaza de Felipe Álvarez Gadea, 28100 Alcobendas, Madrid, Spain',
 'price_level': 1.0,
 'place': 'karaoke',
 'CP': '28100',
 'latitude': 40.5404831,
 'longitude': -3.6322001,
 'geometry': {'type': 'Point', 'coordinates': [-3.6322001, 40.5404831]},
 'price': '1-15€'}

In [17]:
df1 = pd.DataFrame.from_dict(d_random)
df2 = pd.DataFrame.from_dict(p_random)
df = pd.concat([df1,df2])

In [18]:
#df = df.set_index("name")
df =df.drop(["geometry"], axis = 1)

In [19]:
df = df.drop_duplicates()

In [20]:
df

,_id,name,rating,address,price_level,place,CP,latitude,longitude,price
coordinates,60a0f5a7b68aeec84a076b84,TomeVinos,4.7,"Paseo de la Chopera, 194, 28100 Alcobendas, Ma...",2.0,wine,28100,40.545338,-3.659049,1-15€
coordinates,60a0f5a85976320580e34311,Sala Liverpool Karaoke,3.4,"Plaza de Felipe Álvarez Gadea, 28100 Alcobenda...",1.0,karaoke,28100,40.540483,-3.632200,1-15€


## General

In [21]:
def planes(df_1, df_2, coll_1, coll_2):
    
    intersection = set(list(df_1.CP.unique())).intersection(set(list(df_2.CP.unique())))

    cp = list(intersection)

    cp_random = random.choice(cp)

    plan1 = list(coll_1.find({"CP":str(int(cp_random))}))

    plan2 = list(coll_2.find({"CP":str(int(cp_random))}))
    
    p1_random = random.choice(plan1)
    p2_random = random.choice(plan2)

    df1 = pd.DataFrame.from_dict(p1_random)
    df2 = pd.DataFrame.from_dict(p2_random)
    df = pd.concat([df1,df2])
    
    df = df.set_index("_id")
    df =df.drop(["geometry"], axis = 1)
    df = df.drop_duplicates()
    
    return df

In [22]:
def get_df(df):
    df = df.set_index("name")
    return df.drop(["place", "CP", "latitude", "longitude"], axis=1)

In [23]:
def get_map(df):
    map_1 = Map(location=[list(df.latitude.unique())[0],list(df.longitude.unique())[0]],zoom_start=15)
    for i, row in df.iterrows():
        geom = {
                    "location":[row["latitude"], row["longitude"]],
                    "tooltip" : row["name"]
                }
        icon = Icon(color = "white",
                                prefix = "fa",
                                icon = "trophy",
                                icon_color = "black"
                )


        Marker(**geom,icon = icon ).add_to(map_1)
        
    return map_1

In [24]:
def type_of_plan(plan):
    
    options_team_building = { "option_1" : [df_outdoors, df_restaurants, outdoors, restaurants],
                             "option_2" : [df_leisure, df_restaurants, leisure, restaurants]
                            }

    options_friends = { "option_1" : [df_restaurants, df_drinks, restaurants, drinks],
                        "option_2" : [df_outdoors, df_restaurants, outdoors, restaurants],
                        "option_3" : [df_leisure, df_restaurants, leisure, restaurants],
                        "option_4" : [df_party, df_restaurants, party, restaurants],
                        "option_5" : [df_drinks, df_party, drinks, party],
                        "option_6" : [df_snacks, df_leisure, snacks, leisure],
                        "option_7" : [df_outdoors, df_snacks, outdoors, snacks]
                     }

    options_couple = { "option_1" : [df_restaurants, df_drinks, restaurants, drinks],
                       "option_2" : [df_leisure, df_restaurants, leisure, restaurants],
                       "option_3" : [df_culture, df_restaurants, culture, restaurants],
                       "option_4" : [df_culture, df_snacks, culture, snacks]
                     }

    options_individual = { "option_1" : [df_culture, df_restaurants, culture, restaurants],
                           "option_2" : [df_culture, df_snacks, culture, snacks]
                         }

    options_family = {  "option_1" : [df_culture, df_restaurants, culture, restaurants],
                        "option_2" : [df_culture, df_snacks, culture, snacks],
                        "option_3" : [df_outdoors, df_restaurants, outdoors, restaurants],
                        "option_4" : [df_outdoors, df_snacks, outdoors, snacks],
                        "option_5" : [df_leisure, df_restaurants, leisure, restaurants],
                        "option_6" : [df_leisure, df_snacks, leisure, snacks]
                     }
    
    if plan == "family":
        options =  options_family
    elif plan == "friends":
        options = options_friend
    elif plan == "individual":
        options = options_individual
    elif plan == "couple":
        options = options_couple
    elif plan == "team building":
        options = options_team_building
     
    opt = random.choice(list(options.keys()))
    args = options[opt]
    arg_1 = args[0]  
    arg_2 = args[1]  
    arg_3 = args[2]  
    arg_4 = args[3]  
    
    return planes(arg_1, arg_2, arg_3, arg_4)

In [25]:
plan = type_of_plan("couple")

In [26]:
get_df(plan)

,rating,address,price_level,price
name,,,,
Duomo's Pizza,3.7,"Calle de Caleruega, 3, 28033 Madrid, Spain",1.0,1-15€
Soccer Bar Cervecería,3.9,"Calle de Caleruega, 1, 28033 Madrid, Spain",2.0,1-15€


In [44]:
map_ = get_map(plan)

In [96]:
def planes_price(max_):
    
    options = {"snacks + restaurant" : [df_snacks, df_restaurants, snacks, restaurants],
     "snacks + leisure activity" : [df_snacks, df_leisure, snacks, leisure],
     "snacks + outdoors activity" : [df_snacks, df_outdoors, snacks, outdoors],
     "snacks + culture" : [df_snacks, df_culture, snacks, culture],
     "snacks + drinks" : [df_snacks, df_drinks, snacks, drinks],
     "snacks + party" : [df_snacks, df_party, snacks, party],
     "restaurant + leisure activity" : [df_restaurants, df_leisure, restaurants, leisure],
     "restaurant + outdoors activity" : [df_restaurants, df_outdoors, restaurants, outdoors],
     "restaurant + culture" : [df_restaurants, df_culture, restaurants, culture],
     "restaurant + drinks" : [df_restaurants, df_drinks, restaurants, drinks],
     "restaurant + party" : [df_restaurants, df_party, restaurants, party],
     "leisure activity + outdoors activity" : [df_leisure, df_outdoors, leisure, outdoors],
     "leisure activity + culture" : [df_leisure, df_culture, leisure, culture],
     "leisure activity + drinks" : [df_leisure, df_drinks, leisure, drinks],
     "leisure activity + party" : [df_leisure, df_party, leisure, party],  
     "outdoors activity + culture" : [df_outdoors, df_culture, outdoors, culture],
     "outdoors activity + drinks" : [df_outdoors, df_drinks, outdoors, drinks],
     "outdoors activity + party" : [df_outdoors, df_party, outdoors, party],
     "culture + drinks" : [df_culture, df_drinks, culture, drinks],
     "culture + party" : [df_culture, df_party, culture, party],
     "drinks + party" : [df_drinks, df_party, drinks, party]
    }
     
    opt = random.choice(list(options.keys()))
    args = options[opt]
    df_1 = args[0]  
    df_2 = args[1]  
    coll_1 = args[2]  
    coll_2 = args[3] 
    
    intersection = set(list(df_1.CP.unique())).intersection(set(list(df_2.CP.unique())))

    cp = list(intersection)

    cp_random = random.choice(cp)

    plan1 = list(coll_1.find({"CP":str(int(cp_random))}))

    plan2 = list(coll_2.find({"CP":str(int(cp_random))}))
    
    p1_random = random.choice(plan1)
    p2_random = random.choice(plan2)

    df1 = pd.DataFrame.from_dict(p1_random)
    df2 = pd.DataFrame.from_dict(p2_random)
    df = pd.concat([df1,df2])
    
    df = df.set_index("_id")
    df =df.drop(["geometry"], axis = 1)
    df = df.drop_duplicates()
    
    prices = []
    for p in list(df.price_level):
        if p == 0.0:
            prices.append(0)
        elif p == 1.0:
            prices.append(15)
        elif p == 2.0:
            prices.append(25)
        elif p == 3.0:
            prices.append(50)
        else: 
            prices.append(150)

    if sum(prices) > max_:
        return planes_price(max_)
    else: 
        return df

In [98]:
def select_plans(max_):
    
    options = {"snacks + restaurant" : [df_snacks, df_restaurants, snacks, restaurants],
     "snacks + leisure activity" : [df_snacks, df_leisure, snacks, leisure],
     "snacks + outdoors activity" : [df_snacks, df_outdoors, snacks, outdoors],
     "snacks + culture" : [df_snacks, df_culture, snacks, culture],
     "snacks + drinks" : [df_snacks, df_drinks, snacks, drinks],
     "snacks + party" : [df_snacks, df_party, snacks, party],
     "restaurant + leisure activity" : [df_restaurants, df_leisure, restaurants, leisure],
     "restaurant + outdoors activity" : [df_restaurants, df_outdoors, restaurants, outdoors],
     "restaurant + culture" : [df_restaurants, df_culture, restaurants, culture],
     "restaurant + drinks" : [df_restaurants, df_drinks, restaurants, drinks],
     "restaurant + party" : [df_restaurants, df_party, restaurants, party],
     "leisure activity + outdoors activity" : [df_leisure, df_outdoors, leisure, outdoors],
     "leisure activity + culture" : [df_leisure, df_culture, leisure, culture],
     "leisure activity + drinks" : [df_leisure, df_drinks, leisure, drinks],
     "leisure activity + party" : [df_leisure, df_party, leisure, party],  
     "outdoors activity + culture" : [df_outdoors, df_culture, outdoors, culture],
     "outdoors activity + drinks" : [df_outdoors, df_drinks, outdoors, drinks],
     "outdoors activity + party" : [df_outdoors, df_party, outdoors, party],
     "culture + drinks" : [df_culture, df_drinks, culture, drinks],
     "culture + party" : [df_culture, df_party, culture, party],
     "drinks + party" : [df_drinks, df_party, drinks, party]
    }
     
    opt = random.choice(list(options.keys()))
    args = options[opt]
    arg_1 = args[0]  
    arg_2 = args[1]  
    arg_3 = args[2]  
    arg_4 = args[3]  
    
    return planes_price(arg_1, arg_2, arg_3, arg_4,max_)
    

In [97]:
planes(df_snacks, df_culture, snacks, culture,10)

'Not found'

In [55]:
prices = []
for p in list(df.price_level):
    if p == 0.0:
        prices.append(0)
    elif p == 1.0:
        prices.append(15)
    elif p == 2.0:
        prices.append(25)
    elif p == 3.0:
        prices.append(50)
    else: 
        prices.append(150)

In [58]:
sum(prices)

40

In [59]:
max_ = 60

In [60]:
call = planes(df_snacks, df_culture, snacks, culture)

In [77]:
prices = [100,150]

In [78]:
def plan_check(max_):
    try:
        if sum(prices) > max_:
            return call
        else: 
            return df
    except:
        "Not found"

In [79]:
plan_check(10)

,name,rating,address,price_level,place,CP,latitude,longitude,price
_id,,,,,,,,,
60a0f5a8709918f94d5e6f16,Juicy Avenue,4.1,"CC Moraleja Green, Av. de Europa, 13, 28108 Ma...",1.0,juices,28108,40.518858,-3.655867,15-25€
60a0f5a7469adda97be6c625,Kinépolis Madrid Diversia,4.2,"C.C. Heron Diversia, Av. de Bruselas, 21, 2810...",1.0,cinema,28108,40.530082,-3.639672,25-50€
